# Notebook That Calibrate ADC DC and AC Effects

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Vref = 3.3 #Reference Voltage of ADC on ESP32

### Two-Point Calibration

In [ ]:
V_low = 0.1*Vref
V_high = 0.9*Vref
Measured_Code_low  = 278
Measured_Code_High = 3780
Expected_Code_low = 409
Expected_Code_High = 3686

In [ ]:
ideal_slope = (Expected_Code_High-Expected_Code_low)/(V_high - V_low)
ideal_slope

In [ ]:
non_ideal_slope = (Measured_Code_High-Measured_Code_low)/(V_high-V_low)
non_ideal_slope

In [ ]:
def calibrate(raw_value):
    return int(round((raw_value +159)*ideal_slope/non_ideal_slope,0))

In [ ]:
print(calibrate(278))

In [ ]:
real = [278,3780]
ideal = [409,3686]
vin = [V_low, V_high]
plt.plot(vin,real,"-", label = "non-ideal",marker = "s", markersize = 5)
plt.plot(vin,ideal,"-", label = "ideal", marker = 10, markersize = 5)
plt.xlabel ("$V_{in}(v)$")
plt.ylabel ("ADC Code")
plt.title("Ideal and Non-ideal transfer functions")
plt.legend();

### Collecting Data for Linear Regression

In [ ]:
input_voltages = np.array([0.00,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.1,3.2,3.3])
ADC_codes = np.array([0,0,0,111,235,358,488,612,731,853,994,1113,1374,1636,1877,2134,2384,2642,2892,3173,3465,3886,4069,4095,4095])

In [ ]:
plt.scatter(input_voltages,ADC_codes);

### Callecting Data for Kalman Filter

In [ ]:
input_Volatage = 2.0 #
true_ADC_Code = 2482
measured_codes = np.array([2392,2393,2383,2386,2387,2383,2367,2384,2384,2383,2394,2282,2395,2395,2387,2384,2393,2389,2390,2378,2382,2384,2385
])
len(measured_codes)